<a href="https://colab.research.google.com/github/shirinlatifi/pe-project/blob/master/buildNet%2CFNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd /content/drive/My Drive/ANN

/content/drive/My Drive/ANN


In [3]:
!git clone https://github.com/angular/angular-phonecat.git

fatal: destination path 'angular-phonecat' already exists and is not an empty directory.


In [4]:
cd /content/drive/My Drive/pybrain

/content/drive/My Drive/pybrain


In [5]:
!python setup.py install

running install
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer, pypa/build or
        other standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/cmd.py:66: EasyInstallDeprecationWarning: easy_install command is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` and ``easy_install``.
        Instead, use pypa/build, pypa/installer, pypa/build or
        other standards-based t

In [6]:
from pybrain.tools.shortcuts import buildNetwork
from pybrain.structure import TanhLayer
from pybrain.datasets import SupervisedDataSet
from pybrain.supervised.trainers import BackpropTrainer
import pandas as pd

In [7]:
cd /content/drive/My Drive

/content/drive/My Drive


In [8]:
df = pd.read_excel('oil SG.xlsx')

In [9]:
df

,T,P,Rs,Sgo
0,140.0,2551.0,683.000000,0.896307
1,163.0,2766.0,706.000000,0.869913
2,150.0,2110.0,571.410000,0.852564
3,165.0,2181.0,591.700000,0.854365
4,150.0,2158.0,643.530000,0.846039
...,...,...,...,...
501,205.0,830.0,1.299484,0.772170
502,205.0,625.0,1.280984,0.777714
503,205.0,425.0,1.262065,0.782922
504,205.0,229.0,1.235861,0.790140


#data cleaning

In [10]:
import numpy as np

In [11]:
df.drop('Rs', axis=1, inplace=True)

In [12]:
def fillnanP(col):
    if pd.isnull(col):
      return (df['P']!=np.NAN).mean()
    else:
      return col

In [13]:
def fillnanT(col):
    if pd.isnull(col):
      return (df['T']!=np.NAN).mean()
    else:
      return col

In [14]:
df['P'] = df['P'].apply(fillnanP)

In [15]:
df['T'] = df['T'].apply(fillnanT)

In [16]:
df.dropna(inplace=True)

In [ ]:
df

In [ ]:
df.isnull()

#scaling dataset and cleaning

In [20]:
from sklearn.preprocessing import StandardScaler

In [21]:
scaler = StandardScaler()

In [22]:
scaler.fit(df.drop('Sgo', axis=1))

StandardScaler()

In [23]:
scaled_X = scaler.transform(df.drop('Sgo', axis=1))

In [ ]:
scaled_X

In [25]:
scaled_X.shape

(480, 2)

In [26]:
type(scaled_X)

numpy.ndarray

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 480 entries, 0 to 505
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   T       480 non-null    float64
 1   P       480 non-null    float64
 2   Sgo     480 non-null    float64
dtypes: float64(3)
memory usage: 15.0 KB


In [29]:
df['T'] = scaled_X[:,0]

In [30]:
df['P'] = scaled_X[:,1]

In [31]:
df

,T,P,Sgo
0,-0.256286,1.328867,0.896307
1,0.101273,1.491641,0.869913
2,-0.100826,0.994989,0.852564
3,0.132365,1.048743,0.854365
4,-0.100826,1.031330,0.846039
...,...,...,...
501,0.754205,0.025913,0.772170
502,0.754205,-0.129291,0.777714
503,0.754205,-0.280709,0.782922
504,0.754205,-0.429099,0.790140


# test train split

In [32]:
data = df.values

In [33]:
data

array([[-0.25628573,  1.32886662,  0.89630709],
       [ 0.10127256,  1.49164119,  0.8699127 ],
       [-0.10082561,  0.99498947,  0.85256372],
       ...,
       [ 0.75420509, -0.28070893,  0.78292162],
       [ 0.75420509, -0.42909878,  0.79014025],
       [ 0.75420509, -0.59134339,  0.8864    ]])

In [34]:
train_data = data[:,0:2]
train_output = data[:,2]


In [ ]:
train_data

In [38]:
type(train_data)

numpy.ndarray

In [39]:
train_data.shape

(480, 2)

In [ ]:
train_output

In [37]:
train_output.shape

(480,)

In [40]:
type(train_output)

numpy.ndarray

#Create a network with two inputs, three hidden, and one output

In [41]:
nn = buildNetwork(2, 3, 1, bias=True, hiddenclass=TanhLayer)

#Create a dataset that matches network input and output sizes

In [42]:
gate = SupervisedDataSet(2, 1)

In [43]:
len(train_output)

480

In [44]:
len(train_data)

480

In [45]:
# Add input and target values to dataset
for i in range(0, len(train_output)) :
   gate.addSample(train_data[i], train_output[i])

In [46]:
#Training the network with dataset
trainer = BackpropTrainer(nn, gate)

In [47]:
# will run the loop 1000 times to train it.
for epoch in range(1000):
   trainer.train()

In [48]:
trainer

<BackpropTrainer 'BackpropTrainer-9'>

In [49]:
trainer.testOnData(dataset=gate, verbose = True)


Testing on data:
('out:    ', '[0.814 ]')
('correct:', '[0.896 ]')
error:  0.00335287
('out:    ', '[0.808 ]')
('correct:', '[0.87  ]')
error:  0.00189478
('out:    ', '[0.816 ]')
('correct:', '[0.853 ]')
error:  0.00066149
('out:    ', '[0.81  ]')
('correct:', '[0.854 ]')
error:  0.00096554
('out:    ', '[0.816 ]')
('correct:', '[0.846 ]')
error:  0.00046578
('out:    ', '[0.81  ]')
('correct:', '[0.842 ]')
error:  0.00049365
('out:    ', '[0.82  ]')
('correct:', '[0.854 ]')
error:  0.00057816
('out:    ', '[0.799 ]')
('correct:', '[0.873 ]')
error:  0.00269833
('out:    ', '[0.799 ]')
('correct:', '[0.871 ]')
error:  0.00258954
('out:    ', '[0.821 ]')
('correct:', '[0.912 ]')
error:  0.00414148
('out:    ', '[0.802 ]')
('correct:', '[0.916 ]')
error:  0.00649555
('out:    ', '[0.802 ]')
('correct:', '[0.921 ]')
error:  0.00700766
('out:    ', '[0.823 ]')
('correct:', '[0.902 ]')
error:  0.00312183
('out:    ', '[0.805 ]')
('correct:', '[0.909 ]')
error:  0.00550345
('out:    ', '[0

0.0007458500030776044

#FNN

In [50]:
#Adding Modules and Connections

from pybrain.structure import FeedForwardNetwork
from pybrain.structure import LinearLayer, SigmoidLayer
from pybrain.structure import FullConnection


In [51]:
network = FeedForwardNetwork()

In [52]:
#creating layer for input => 2 , hidden=> 3 and output=>1
inputLayer = LinearLayer(2)
hiddenLayer = SigmoidLayer(3)
outputLayer = LinearLayer(1)

In [53]:
#adding the layer to feedforward network
network.addInputModule(inputLayer)
network.addModule(hiddenLayer)
network.addOutputModule(outputLayer)

In [54]:
#Create connection between input ,hidden and output
input_to_hidden = FullConnection(inputLayer, hiddenLayer)
hidden_to_output = FullConnection(hiddenLayer, outputLayer)

In [55]:
#add connection to the network
network.addConnection(input_to_hidden)
network.addConnection(hidden_to_output)
network.sortModules()

In [56]:
print(network)

FeedForwardNetwork-16
   Modules:
    [<LinearLayer 'LinearLayer-13'>, <SigmoidLayer 'SigmoidLayer-17'>, <LinearLayer 'LinearLayer-18'>]
   Connections:
    [<FullConnection 'FullConnection-14': 'LinearLayer-13' -> 'SigmoidLayer-17'>, <FullConnection 'FullConnection-15': 'SigmoidLayer-17' -> 'LinearLayer-18'>]



In [59]:
#Training Network Using Optimization Algorithms
from pybrain.datasets import SupervisedDataSet
from pybrain.optimization.populationbased.ga import GA
from pybrain.tools.shortcuts import buildNetwork

In [60]:
import numpy as np

#buildNet

In [63]:
ds = SupervisedDataSet(2,1)

In [64]:
for i in range(0, len(train_output)) :
   ds.addSample(train_data[i], train_output[i])

In [65]:
print(ds)

input: dim(510, 2)
[[-0.25628573  1.32886662]
 [ 0.10127256  1.49164119]
 [-0.10082561  0.99498947]
 [ 0.13236458  1.04874293]
 [-0.10082561  1.03132984]
 [ 0.13236458  1.06994148]
 [-0.87812624  1.90047035]
 [ 1.37604559  2.38425153]
 [ 1.37604559  2.38273734]
 [-0.56720599  1.30236843]
 [ 0.67647502  1.57643539]
 [ 0.67647502  1.57794957]
 [-0.56720599  1.20924623]
 [ 0.44328483  1.47574228]
 [ 0.44328483  1.47725646]
 [-2.41718149 -0.60171553]
 [-0.3340158  -0.60171553]
 [ 1.31386154 -0.60171553]
 [ 1.31386154 -0.60171553]
 [ 0.5210149  -0.60171553]
 [-0.64493605 -0.60171553]
 [ 0.5210149  -0.60171553]
 [ 1.1428554  -0.60171553]
 [ 1.70251186 -0.60171553]
 [-0.76930415 -0.60171553]
 [-2.41718149 -0.60171553]
 [-0.75375814 -0.60171553]
 [-0.80039618 -0.60171553]
 [-0.81594219 -0.60171553]
 [-0.80039618 -0.60171553]
 [-0.72266611 -0.60171553]
 [-1.09577042 -0.60171553]
 [-0.78485016 -0.60171553]
 [-1.09577042 -0.60171553]
 [-0.76930415 -0.60171553]
 [-0.80039618 -0.60171553]
 [-0.8003

In [66]:
net = buildNetwork(2, 3, 1)
ga = GA(ds.evaluateModuleMSE, net, minimize=True)

In [67]:
for i in range(100):
    net = ga.learn(0)[0]

In [68]:
print(net.activate([0,0]))

[-1.33890452]
